In [ ]:
!pip install sympy sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from sentence_transformers import SentenceTransformer, util
import sympy
import re

# Load a sentence transformer to help with prompt classification (you can improve this later)
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Define keywords that suggest symbolic reasoning
symbolic_keywords = [
    "solve", "calculate", "equation", "expression", "derivative", "integral",
    "what is x", "find x", "math", "algebra"
]

In [ ]:
def classify_prompt(prompt: str):
    prompt_lower = prompt.lower()

    # Heuristic-based keyword detection (lightweight controller logic)
    for keyword in symbolic_keywords:
        if keyword in prompt_lower:
            return "symbolic"

    # Can be extended with vector similarity using sentence-transformers
    return "base"

In [ ]:
from sympy import symbols, Eq, solve, sympify, diff, integrate
import re

x = symbols('x')

def insert_implicit_multiplication(expr: str) -> str:
    # Replace patterns like 4x with 4*x
    expr = re.sub(r'(\d)([a-zA-Z])', r'\1*\2', expr)
    # Replace patterns like x2 with x*2 (optional)
    expr = re.sub(r'([a-zA-Z])(\d)', r'\1*\2', expr)
    return expr

def symbolic_reasoning(prompt: str):
    prompt = prompt.lower().strip().replace('^', '**')
    prompt = re.sub(r'[?]', '', prompt)

    try:
        if "derivative" in prompt:
            expr_match = re.search(r"derivative of (.+)", prompt)
            if not expr_match:
                return "Couldn't find expression to differentiate."
            expr = insert_implicit_multiplication(expr_match.group(1).strip())
            expr_sympy = sympify(expr)
            result = diff(expr_sympy, x)
            return f"The derivative of {expr} is: {result}"

        if "integral" in prompt:
            expr_match = re.search(r"integral of (.+)", prompt)
            if not expr_match:
                return "Couldn't find expression to integrate."
            expr = insert_implicit_multiplication(expr_match.group(1).strip())
            expr_sympy = sympify(expr)
            result = integrate(expr_sympy, x)
            return f"The integral of {expr} is: {result}"

        eq_match = re.search(r"solve (.+)=(.+)", prompt)
        if eq_match:
            left_expr = insert_implicit_multiplication(eq_match.group(1).strip())
            right_expr = insert_implicit_multiplication(eq_match.group(2).strip())
            left = sympify(left_expr)
            right = sympify(right_expr)
            equation = Eq(left, right)
            result = solve(equation, x)
            return f"The solution is: x = {result}"

        return "Sorry, I couldn't parse the symbolic expression."

    except Exception as e:
        return f"Symbolic Reasoning Error: {str(e)}"

In [ ]:
def process_prompt(prompt: str):
    task = classify_prompt(prompt)
    print(f"Controller Decision: {task.upper()} HEAD")

    if task == "symbolic":
        return symbolic_reasoning(prompt)
    else:
        return "Use Groq LLM here (not implemented yet)"

In [ ]:
test_prompts = [
    "Solve 2*x + 5 = 11",
    "What is the derivative of x^2 + 3*x?",
    "Tell me a story about AI"
]

for p in test_prompts:
    print("\nPrompt:", p)
    print("Output:", process_prompt(p))


Prompt: Solve 2*x + 5 = 11
Controller Decision: SYMBOLIC HEAD
Output: The solution is: x = [3]

Prompt: What is the derivative of x^2 + 3*x?
Controller Decision: SYMBOLIC HEAD
Output: The derivative of x**2 + 3*x is: 2*x + 3

Prompt: Tell me a story about AI
Controller Decision: BASE HEAD
Output: Use Groq LLM here (not implemented yet)


In [ ]:
!pip install openai

In [ ]:
import os
import openai

# Replace YOUR_GROQ_API_KEY with your actual key
os.environ["GROQ_API_KEY"] = "PUT_YOUR_GROQ_API_KEY_HERE"
openai.api_key = os.environ["GROQ_API_KEY"]
openai.api_base = "https://api.groq.com/openai/v1"

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

def call_groq_llm(prompt: str, model="llama-3.3-70b-versatile"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"[Groq API Error] {str(e)}"

In [ ]:
def process_prompt(prompt: str):
    task = classify_prompt(prompt)
    print(f"Controller Decision: {task.upper()} HEAD")

    if task == "symbolic":
        return symbolic_reasoning(prompt)
    else:
        return call_groq_llm(prompt)

In [ ]:
test_prompts = [
    "Solve 2*x + 7 = 17",
    "Tell me about the future of artificial intelligence",
    "What is the derivative of x^3 + 4x"
]

for p in test_prompts:
    print("\n---")
    print("Prompt:", p)
    print("Output:", process_prompt(p))


---
Prompt: Solve 2*x + 7 = 17
Controller Decision: SYMBOLIC HEAD
Output: The solution is: x = [5]

---
Prompt: Tell me about the future of artificial intelligence
Controller Decision: BASE HEAD
Output: The future of artificial intelligence (AI) is vast and rapidly evolving. As AI technology continues to advance, we can expect significant impacts on various aspects of our lives, from workforce and economy to healthcare, education, and entertainment. Here are some potential developments and trends that may shape the future of AI:

**Short-term (2025-2035):**

1. **Increased automation**: AI-powered automation will continue to transform industries, making processes more efficient and reducing the need for human labor in certain sectors.
2. **Improved natural language processing (NLP)**: AI-driven NLP will enable more accurate and human-like interactions with virtual assistants, chatbots, and voice-activated devices.
3. **Advances in computer vision**: AI-powered computer vision will imp

In [ ]:
def logical_reasoning(prompt: str):
    try:
        cot_prompt = f"Let's think step by step.\n{prompt}"
        return call_groq_llm(cot_prompt)
    except Exception as e:
        return f"Logical Reasoning Error: {str(e)}"

In [ ]:
graph_keywords = ["parent of", "child of", "connected to", "ancestor", "descendant", "graph", "path from"]

In [ ]:
def controller(prompt: str) -> str:
    prompt = prompt.lower()

    symbolic_keywords = ["solve", "derivative", "integral", "differentiate", "equation"]
    logic_keywords = ["if", "then", "how many", "more than", "less than", "each", "every", "either", "neither", "both", "logic", "reason", "given"]

    if any(keyword in prompt for keyword in symbolic_keywords):
        return "SYMBOLIC HEAD"
    elif any(keyword in prompt for keyword in logic_keywords):
        return "LOGICAL HEAD"
        # Add after symbolic & logic detection
    elif any(keyword in prompt for keyword in graph_keywords):
        return "GRAPH HEAD"
    else:
        return "BASE HEAD"

In [ ]:
def process_prompt(prompt: str) -> str:
    head = controller(prompt)

    if head == "SYMBOLIC HEAD":
        output = symbolic_reasoning(prompt)
    elif head == "LOGICAL HEAD":
        output = logical_reasoning(prompt)
    elif head == "GRAPH HEAD":
        output = graph_reasoning(prompt)
    else:
        output = call_groq_llm(prompt)

    return f"""---
Prompt: {prompt}
Controller Decision: {head}
Output: {output}
---"""

In [ ]:
process_prompt("If Alice is taller than Bob, and Bob is taller than Charlie, who is the shortest?")

"---\nPrompt: If Alice is taller than Bob, and Bob is taller than Charlie, who is the shortest?\nController Decision: LOGICAL HEAD\nOutput: Let's break it down step by step:\n\n1. Alice is taller than Bob.\n2. Bob is taller than Charlie.\n\nFrom these two statements, we can conclude that the order of height is:\n1. Alice (tallest)\n2. Bob (middle)\n3. Charlie (shortest)\n\nSo, Charlie is the shortest.\n---"

In [ ]:
test_prompts = [
    "Solve 2*x + 7 = 17",
    "Tell me about the future of artificial intelligence",
    "What is the derivative of x^3 + 4x",
    "If Alice is taller than Bob, and Bob is taller than Charlie, who is the shortest?"
    "Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?"
]

for p in test_prompts:
    print("\n---")
    print("Prompt:", p)
    print("Output:", process_prompt(p))


---
Prompt: Solve 2*x + 7 = 17
Output: ---
Prompt: Solve 2*x + 7 = 17
Controller Decision: SYMBOLIC HEAD
Output: The solution is: x = [5]
---

---
Prompt: Tell me about the future of artificial intelligence
Output: ---
Prompt: Tell me about the future of artificial intelligence
Controller Decision: LOGICAL HEAD
Output: The future of artificial intelligence (AI) is a vast and rapidly evolving topic. To break it down, let's consider the following key areas:

1. **Advancements in Machine Learning**: Expect significant improvements in machine learning algorithms, enabling AI systems to learn from data more efficiently and make better decisions. This could lead to more accurate predictions, improved natural language processing, and enhanced computer vision.

2. **Increased Adoption in Industries**: AI will become more prevalent in various sectors, such as:
	* Healthcare: AI-assisted diagnosis, personalized medicine, and robotic surgery.
	* Finance: AI-powered trading, risk management, and 

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx

In [ ]:
def graph_reasoning(prompt: str) -> str:
    try:
        # Build a directed graph from the relationships in prompt
        G = nx.DiGraph()

        # Extract basic relationships (e.g., "A is the parent of B")
        relations = re.findall(r'([A-Z][a-z]*) is the (\w+) of ([A-Z][a-z]*)', prompt)
        if not relations:
            return "Graph Reasoning Error: Couldn't extract relationships."

        for subject, relation, obj in relations:
            G.add_edge(subject, obj, relation=relation)

        # Detect reasoning query
        if "ancestor" in prompt:
            person = re.search(r'ancestor of ([A-Z][a-z]*)', prompt)
            if person:
                target = person.group(1)
                ancestors = nx.ancestors(G, target)
                return f"Ancestors of {target}: {', '.join(ancestors) if ancestors else 'None'}"

        if "descendant" in prompt:
            person = re.search(r'descendant of ([A-Z][a-z]*)', prompt)
            if person:
                target = person.group(1)
                descendants = nx.descendants(G, target)
                return f"Descendants of {target}: {', '.join(descendants) if descendants else 'None'}"

        if "path from" in prompt:
            people = re.search(r'path from ([A-Z][a-z]*) to ([A-Z][a-z]*)', prompt)
            if people:
                src, dst = people.groups()
                try:
                    path = nx.shortest_path(G, src, dst)
                    return f"Path from {src} to {dst}: {' → '.join(path)}"
                except nx.NetworkXNoPath:
                    return f"No path from {src} to {dst}."

        return "Graph built, but no reasoning query detected."

    except Exception as e:
        return f"Graph Reasoning Error: {str(e)}"

In [ ]:
import json
from datetime import datetime

In [ ]:
process_prompt("Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?")

'---\nPrompt: Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?\nController Decision: GRAPH HEAD\nOutput: Graph built, but no reasoning query detected.\n---'

In [ ]:
def log_interaction(prompt: str, head: str, output: str, log_file: str = "logs.jsonl"):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "prompt": prompt,
        "controller_decision": head,
        "output": output
    }
    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(json.dumps(log_entry) + "\n")
    except Exception as e:
        print(f"Logging Error: {str(e)}")

In [ ]:
def process_prompt(prompt: str) -> str:
    head = controller(prompt)

    if head == "SYMBOLIC HEAD":
        output = symbolic_reasoning(prompt)
    elif head == "LOGICAL HEAD":
        output = logical_reasoning(prompt)
    elif head == "GRAPH HEAD":
        output = graph_reasoning(prompt)
    else:
        output = call_groq_llm(prompt)

    # Log interaction
    log_interaction(prompt, head, output)

    return f"""---
Prompt: {prompt}
Controller Decision: {head}
Output: {output}
---"""

In [ ]:
test_prompts = [
    "Solve 2*x + 7 = 17",
    "Tell me about the future of artificial intelligence",
    "What is the derivative of x^3 + 4x",
    "If Alice is taller than Bob, and Bob is taller than Charlie, who is the shortest?"
    "Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?"
]

for p in test_prompts:
    print("\n---")
    print("Prompt:", p)
    print("Output:", process_prompt(p))


---
Prompt: Solve 2*x + 7 = 17
Output: ---
Prompt: Solve 2*x + 7 = 17
Controller Decision: SYMBOLIC HEAD
Output: The solution is: x = [5]
---

---
Prompt: Tell me about the future of artificial intelligence
Output: ---
Prompt: Tell me about the future of artificial intelligence
Controller Decision: LOGICAL HEAD
Output: The future of artificial intelligence (AI) holds tremendous promise and potential. Let's break it down into several key areas:

1. **Increased Adoption**: AI will become more pervasive in various industries, such as healthcare, finance, education, and transportation. We can expect to see more AI-powered tools, systems, and applications that enhance efficiency, productivity, and decision-making.

2. **Advancements in Machine Learning**: Machine learning, a subset of AI, will continue to advance, enabling computers to learn from data, improve their performance, and make more accurate predictions. This will lead to significant breakthroughs in areas like natural language p

In [ ]:
!cat logs.jsonl

{"timestamp": "2025-06-18T03:42:02.123357", "prompt": "Solve 2*x + 7 = 17", "controller_decision": "SYMBOLIC HEAD", "output": "The solution is: x = [5]"}
{"timestamp": "2025-06-18T03:42:04.503461", "prompt": "Tell me about the future of artificial intelligence", "controller_decision": "LOGICAL HEAD", "output": "The future of artificial intelligence (AI) holds tremendous promise and potential. Let's break it down into several key areas:\n\n1. **Increased Adoption**: AI will become more pervasive in various industries, such as healthcare, finance, education, and transportation. We can expect to see more AI-powered tools, systems, and applications that enhance efficiency, productivity, and decision-making.\n\n2. **Advancements in Machine Learning**: Machine learning, a subset of AI, will continue to advance, enabling computers to learn from data, improve their performance, and make more accurate predictions. This will lead to significant breakthroughs in areas like natural language proces

In [ ]:
def controller(prompt: str) -> tuple[str, str]:
    symbolic_keywords = ["solve", "equation", "derivative", "integrate", "simplify"]
    logic_keywords = ["if", "then", "must be", "true or false", "all", "some", "none"]
    graph_keywords = ["parent of", "child of", "connected to", "ancestor", "descendant", "path from"]

    for keyword in symbolic_keywords:
        if keyword in prompt.lower():
            return "SYMBOLIC HEAD", f"Matched keyword: '{keyword}' → Symbolic reasoning"

    for keyword in logic_keywords:
        if keyword in prompt.lower():
            return "LOGICAL HEAD", f"Matched keyword: '{keyword}' → Logical reasoning"

    for keyword in graph_keywords:
        if keyword in prompt.lower():
            return "GRAPH HEAD", f"Matched keyword: '{keyword}' → Graph reasoning"

    return "BASE HEAD", "No keyword match → Using base LLM (Groq)"

In [ ]:
def process_prompt(prompt: str) -> str:
    head, explanation = controller(prompt)

    if head == "SYMBOLIC HEAD":
        output = symbolic_reasoning(prompt)
    elif head == "LOGICAL HEAD":
        output = logical_reasoning(prompt)
    elif head == "GRAPH HEAD":
        output = graph_reasoning(prompt)
    else:
        output = call_groq_llm(prompt)

    # Log everything (now includes explanation)
    log_interaction(prompt, head, output)

    return f"""---
Prompt: {prompt}
Controller Decision: {head}
Reason: {explanation}
Output: {output}
---"""

In [ ]:
print(process_prompt("If it rains, then the ground gets wet. It is raining. Is the ground wet?"))

---
Prompt: If it rains, then the ground gets wet. It is raining. Is the ground wet?
Controller Decision: LOGICAL HEAD
Reason: Matched keyword: 'if' → Logical reasoning
Output: To answer this question, let's break it down step by step:

1. If it rains, then the ground gets wet. (This is the given condition)
2. It is raining. (This is the current situation)

Using the given condition, we can conclude that since it is raining, the ground should get wet.

Therefore, the answer is: Yes, the ground is wet.
---


In [ ]:
test_prompts = [
    "Solve 2*x + 7 = 17",
    "Tell me about the future of artificial intelligence",
    "What is the derivative of x^3 + 4x",
    "If Alice is taller than Bob, and Bob is taller than Charlie, who is the shortest?",
    "Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?"
]

for p in test_prompts:
    print("\n---")
    print("Prompt:", p)
    print("Output:", process_prompt(p))


---
Prompt: Solve 2*x + 7 = 17
Output: ---
🔍 Highlighted Prompt:
Solve 2*x + 7 = 17

🧠 Controller Decision: SYMBOLIC HEAD
📝 Reason: Matched keyword: 'solve' → Symbolic reasoning

📤 Output:
The solution is: x = [5]
---

---
Prompt: Tell me about the future of artificial intelligence
Output: ---
🔍 Highlighted Prompt:
Tell me about the future of artificial intelligence

🧠 Controller Decision: LOGICAL HEAD
📝 Reason: Matched keyword: 'if' → Logical reasoning

📤 Output:
The future of artificial intelligence (AI) is a vast and rapidly evolving field. Let's break it down step by step:

**Current State:**
AI has already made significant progress in various areas, such as:

1. **Machine Learning (ML)**: AI systems can learn from data and improve their performance over time.
2. **Natural Language Processing (NLP)**: AI can understand and generate human-like language.
3. **Computer Vision**: AI can interpret and understand visual data from images and videos.

**Short-Term Future (2025-2030):**
In

In [ ]:
symbolic_keywords = ["solve", "equation", "derivative", "integrate", "simplify"]
logic_keywords = ["if", "then", "must be", "true or false", "all", "some", "none"]
graph_keywords = ["parent of", "child of", "connected to", "ancestor", "descendant", "path from"]

In [ ]:
!pip install termcolor

In [ ]:
from termcolor import colored  # Install with: pip install termcolor

def highlight_prompt(prompt: str) -> str:
    words = prompt.split()
    highlighted = []

    for word in words:
        lw = word.lower().strip(",.?!")
        if lw in symbolic_keywords:
            highlighted.append(colored(word, "green", attrs=["bold"]))
        elif lw in logic_keywords:
            highlighted.append(colored(word, "blue", attrs=["bold"]))
        elif lw in graph_keywords:
            highlighted.append(colored(word, "magenta", attrs=["bold"]))
        else:
            highlighted.append(word)

    return " ".join(highlighted)

In [ ]:
def process_prompt(prompt: str) -> str:
    head, explanation = controller(prompt)

    if head == "SYMBOLIC HEAD":
        output = symbolic_reasoning(prompt)
    elif head == "LOGICAL HEAD":
        output = logical_reasoning(prompt)
    elif head == "GRAPH HEAD":
        output = graph_reasoning(prompt)
    else:
        output = call_groq_llm(prompt)

    # Log it
    log_interaction(prompt, head, output)

    # Highlight prompt tokens
    highlighted = highlight_prompt(prompt)

    return f"""---
🔍 Highlighted Prompt:
{highlighted}

🧠 Controller Decision: {head}
📝 Reason: {explanation}

📤 Output:
{output}
---"""

In [ ]:
test_prompts = [
    "Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?",
    "Solve 2*x + 7 = 17",
    "Tell me about the future of artificial intelligence",
    "What is the derivative of x^3 + 4x",
    "If Alice is taller than Bob, and Bob is taller than Charlie, who is the shortest?"
    "Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?"
]

for p in test_prompts:
    print("\n---")
    print("Prompt:", p)
    print("Output:", process_prompt(p))


---
Prompt: Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?
Output: ---
🔍 Highlighted Prompt:
Alice is the parent of Bob. Bob is the parent of Charlie. Who are the ancestors of Charlie?

🧠 Controller Decision: GRAPH HEAD
📝 Reason: Matched keyword: 'parent of' → Graph reasoning

📤 Output:
Graph built, but no reasoning query detected.
---

---
Prompt: Solve 2*x + 7 = 17
Output: ---
🔍 Highlighted Prompt:
Solve 2*x + 7 = 17

🧠 Controller Decision: SYMBOLIC HEAD
📝 Reason: Matched keyword: 'solve' → Symbolic reasoning

📤 Output:
The solution is: x = [5]
---

---
Prompt: Tell me about the future of artificial intelligence
Output: ---
🔍 Highlighted Prompt:
Tell me about the future of artificial intelligence

🧠 Controller Decision: LOGICAL HEAD
📝 Reason: Matched keyword: 'if' → Logical reasoning

📤 Output:
Let's break down the future of artificial intelligence (AI) step by step.

**Current State:**
AI has already made significant progress in recent y